## Extração dados API

#### 1. Instalação das Bibliotecas

In [0]:
pip install translate

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install schedule

Python interpreter will be restarted.
Python interpreter will be restarted.


#### 2. Imports

In [0]:
import requests
import time
from translate import Translator
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.utils import AnalysisException
import schedule

#### 3. Constantes

In [0]:
path = '/Filestore/projeto/dados_raw'

translator = Translator(to_lang="en", from_lang='pt')
api_key = "844629b1a5d44eb7938b174b6285a905"

palavras_chave =  ['outubro rosa', 'DNA', 'doença genética', 'terapia genética']

schema = StructType([
        StructField("source", StructType([
            StructField("id", StringType(), True),
            StructField("name", StringType(), True)
        ]), True),
        StructField("author", StringType(), True),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("url", StringType(), True),
        StructField("urlToImage", StringType(), True),
        StructField("publishedAt", StringType(), True),
        StructField("content", StringType(), True)])

#### 4. Extração e carregamento das notícias

In [0]:
def carregamento(palavras_chave: list[str]):
    searchIn = 'title,description,content'
    news_translation = list(map(translator.translate, palavras_chave))

    df_articles = spark.createDataFrame([], schema)

    for noticia in set(palavras_chave + news_translation):
        url = ('https://newsapi.org/v2/everything?'
        f'q=("{noticia}")&'
        f'searchIn={searchIn}&'
        'sortBy=relevancy&'
        f'apiKey={api_key}')

        response = requests.get(url).json()
        if response['status'] == 'ok':
                df_articles = df_articles.union(spark.createDataFrame(response['articles'], schema))
    return df_articles

def processamento_raw(df):
  df = df.dropDuplicates(['url'])
  df = df.withColumn('data_processamento', current_timestamp())
  return df

def save_data(df):
  df.write.format('delta').mode('append').parquet(path)
  print(f"{df.count()} novas notícias foram persistidas")
  
  
def obter_dados_nao_salvos(df):
  try:
    df_leitura = spark.read.parquet(path)
    df_left_join = df.alias("df1").join(df_leitura.alias("df2"), on="url", how="left")
    left_exclude = df_left_join.filter(col("df2.url").isNull()).select("df1.*")
    return left_exclude
  except AnalysisException:
    return df
  
  


In [0]:
def proc_lotes():
    print("O processo começou")
    df = carregamento(palavras_chave)
    df_processada = processamento_raw(df)
    df_novas_noticias = obter_dados_nao_salvos(df_processada)
    save_data(df_novas_noticias)

In [0]:
# schedule.every().hour.do(proc_lotes)          #executa a cada uma hora
schedule.every().minute.do(proc_lotes)        #executa a cada um minuto. Apenas para teste
while True:
    schedule.run_pending()
    time.sleep(1)

O processo começou
420 novas notícias foram persistidas
O processo começou
3 novas notícias foram persistidas
